In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
import time

2024-03-06 19:53:07.013396: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.12.0'

In [3]:
from tensorflow.keras.utils import to_categorical as labelEncoding 

T = 15 # terminus_length

X1 = np.load('bpf-740.npy')


X1 = X1[:,0:T,:]


Y  = [1 for _ in range(376)]
Y += [0 for _ in range(364)]

Y = labelEncoding(Y, dtype=int)

print(X1.shape)
print(Y.shape)

(740, 15, 20)
(740, 15, 31)
(740, 15, 20)
(740, 2)


In [4]:
print(X1.shape)
print(X1[0].shape)
X1[0]

(740, 15, 20)
(15, 20)


array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1,

In [7]:
# Deep Neural Networks:
import tensorflow as tf; print('We\'re using TF-{}.'.format(tf.__version__))
# import keras; print('We\'re using Keras-{}.'.format(keras.__version__))
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten, BatchNormalization,
                                     Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,
                                     LSTM, GRU, Embedding, Bidirectional, Concatenate)
from tensorflow.keras.regularizers import (l1, l2, l1_l2)
from tensorflow.keras.optimizers import (RMSprop, Adam, SGD)
from tensorflow.keras.models import (Sequential, Model)

# Core:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interp
import matplotlib.patches as patches

# Performance:
from sklearn.metrics import (confusion_matrix, classification_report, matthews_corrcoef, precision_score, roc_curve, auc)
from sklearn.model_selection import (StratifiedKFold, KFold, train_test_split)

#Utilities:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical as labelEncoding   # Usages: Y = labelEncoding(Y, dtype=int)
from tensorflow.keras.utils import plot_model   

We're using TF-2.12.0.


In [8]:
def build_discriminator():
    ### Head-1:
    input1 = Input(shape=X1[0].shape)

    x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(input1)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.80)(x)

    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    head1 = Flatten()(x)

    # merge
    merge = Concatenate()([head1])

    output = Dense(units=8, activation='relu', kernel_regularizer=l2(l=0.01))(merge)
    output = BatchNormalization()(output)
    output = Dropout(rate=0.70)(output)

    output = Dense(units=2, activation='softmax')(output)

    return Model(inputs=[input1], outputs=[output])

In [9]:
from Bio.Seq import Seq
from Bio.SeqUtils import ProtParam
from Bio.Align import substitution_matrices
from Bio.SubsMat import MatrixInfo


# Function to convert binary profile feature to amino acid sequence
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        index = row.argmax()
        sequence += amino_acids[index]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row)
    return blosum_matrix

/home/homayoun/anaconda3/envs/acp-proj/lib/python3.11/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [55]:
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        i = 0
        for item in row:
            i += 1
            if item.numpy() == 1:
                sequence += amino_acids[i]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row[:20])
    return blosum_matrix

def bpf_to_blosum_layer(seqs_bpf):
    seqs_blosum = []
    for bpf in seqs_bpf:
        str_seq = bpf_to_sequence(bpf)
        seqs_blosum.append(seq_to_blosum(str_seq))

    print(seqs_blosum)
    return seqs_blosum

def build_generator(latent_dim, output_shape):
    model = Sequential([
        layers.Input(shape=(latent_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(output_shape[0] * output_shape[1], activation='sigmoid'),
        layers.Reshape(output_shape),
        layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=output_shape[1])),
        layers.Lambda(lambda x: tf.cast(x, dtype=tf.int32)),
    ])

    return model

In [58]:
# Define the dimensions
latent_dim = 100  # Dimensionality of the latent space
batch_size = 1
output_shape = (25, 20)  # Desired output shape of the matrix

generator = build_generator(latent_dim, output_shape)

noise = tf.random.normal([1, latent_dim])

generated_matrix = generator(noise, training=False)

generated_matrix

[[[6, -1, 0, 0, 6, 3, -3, -3, 3, 0, -3, -2, 6, 6, 1, -3, 0, -2, 1, -3], [-1, 8, -2, -3, -1, 2, 1, -2, 2, -3, 0, -3, -1, -1, -2, -2, -2, -3, -2, -1], [0, -2, 5, 2, 0, -1, -2, -3, -1, 1, -2, -1, 0, 0, -1, -3, 5, -1, -1, -1], [0, -3, 2, 4, 0, -1, -3, -4, -1, 2, -3, -1, 0, 0, -2, -4, 2, -1, -2, -2], [6, -1, 0, 0, 6, 3, -3, -3, 3, 0, -3, -2, 6, 6, 1, -3, 0, -2, 1, -3], [3, 2, -1, -1, 3, 7, -2, -3, 7, -1, -2, -2, 3, 3, 2, -3, -1, -2, 2, -2], [-3, 1, -2, -3, -3, -2, 6, 0, -2, -3, 0, -3, -3, -3, -4, 0, -2, -3, -4, 0], [-3, -2, -3, -4, -3, -3, 0, 6, -3, -4, -2, -3, -3, -3, -2, 6, -3, -3, -2, -2], [3, 2, -1, -1, 3, 7, -2, -3, 7, -1, -2, -2, 3, 3, 2, -3, -1, -2, 2, -2], [0, -3, 1, 2, 0, -1, -3, -4, -1, 4, -3, -1, 0, 0, -3, -4, 1, -1, -3, -3], [-3, 0, -2, -3, -3, -2, 0, -2, -2, -3, 5, -4, -3, -3, -3, -2, -2, -4, -3, 1], [-2, -3, -1, -1, -2, -2, -3, -3, -2, -1, -4, 9, -2, -2, -2, -3, -1, 9, -2, -3], [6, -1, 0, 0, 6, 3, -3, -3, 3, 0, -3, -2, 6, 6, 1, -3, 0, -2, 1, -3], [6, -1, 0, 0, 6, 3, -3, -3, 3,

InvalidArgumentError: Exception encountered when calling layer 'lambda_73' (type Lambda).

{{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [1,25,20] != values[1].shape = [25,20] [Op:Pack] name: stack

Call arguments received by layer 'lambda_73' (type Lambda):
  • inputs=tf.Tensor(shape=(1, 25, 20), dtype=int32)
  • mask=None
  • training=False

In [ ]:
discriminator = build_discriminator()
discriminator.load_weights('./acp_mhcnn_weights.h5')

In [ ]:
prediction = discriminator.predict([X1[:5,:,:]])
prediction

1/1 [==============================] - 0s 414ms/step


array([[0.22306105, 0.7769389 ],
       [0.2842041 , 0.7157959 ],
       [0.31516826, 0.68483174],
       [0.29036   , 0.70963997],
       [0.30475914, 0.69524086]], dtype=float32)

In [ ]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

# inputs_bpf = []
# inputs_blosum = []
# for seq in fake_seqs:
#     bpf, blosum = convert_input(seq)
#     inputs_bpf.append(bpf[:15])
#     inputs_blosum.append(blosum[:15,:20])

# inputs_bpf = np.array(inputs_bpf)
# inputs_blosum = np.array(inputs_blosum)
# fake_seqs = fake_seqs[:,:15,:]


print(fake_seqs)

# print(X3[:5,:,:].shape)
# print(X3[0])
# print('---------------')
# print(inputs_blosum.shape)
# print(inputs_blosum[0])

prediction = discriminator.predict([fake_seqs[:,:15,:]])
print(prediction)

tf.Tensor(
[[[[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  1]
   [ 1  0  0 ...  0  0  0]]

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  1  0]]

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  1  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 1  0  0 ...  0  0  0]]

  [[ 0  0  1 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  1  0  0]
   ...
   [ 0  1  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]]


 [[[ 5 -1 -1 ... -2 -1 -2]
   [-1  5 -1 ...  0 -2 -1]
   [-1 -1  5 ...  1 -2  2]
   ...
   [-1 -2 -2 ... -3  8 -3]
   [-3 -1  1 ... 

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(disc_output):
    # print(disc_output.shape)
    desired_output = np.zeros_like(disc_output)
    desired_output[:, 1] = 1
    return cross_entropy(desired_output, disc_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
BATCH_SIZE = 32

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, 100])
fake_seqs_sym = generator(noise, training=True)

fake_seqs_sym = fake_seqs_sym[:,:15,:]
fake_output = discriminator([fake_seqs[:,:15,:]])
gen_loss = generator_loss(fake_output)
print(gen_loss)

tf.Tensor(0.6327607, shape=(), dtype=float32)


/home/user/anaconda3/envs/acp-proj/lib/python3.10/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


In [ ]:
@tf.function
def train_step():
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    with tf.GradientTape() as gen_tape:
        fake_seqs_sym = generator(noise, training=True)

        fake_seqs_bpf = fake_seqs_sym[:,:15,:]

        print('test')
        fake_output = discriminator([fake_seqs])
        print(fake_output)
        gen_loss = generator_loss(fake_output)
        print('loss', gen_loss)
            
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    for gradient in gradients_of_generator:
        print("Gradient:", gradient)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

def train(dataset = None, epochs = 50):
    for epoch in range(epochs):
        start = time.time()

        for batch in range(100):
            # train_step(batch[0], batch[1])
            train_step()

        # Generate and save some examples of generated sequences
        if (epoch + 1) % 5 == 0:
            noise = tf.random.normal([num_examples_to_generate, noise_dim])
            generated_sequences = generator(noise, training=False)
            print("Examples of generated sequences at epoch", epoch + 1)
            for i in range(num_examples_to_generate):
                print(bpf_to_sequence(generated_sequences[i].numpy()))

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [ ]:
# Combine datasets X1 and X3
# combined_dataset = tf.data.Dataset.from_tensor_slices((X1, X3)).shuffle(len(X1)).batch(BATCH_SIZE)

# Train the model for 50 epochs
train(epochs=50)

OperatorNotAllowedInGraphError: in user code:

    File "/tmp/ipykernel_384410/1021801238.py", line 6, in train_step  *
        fake_seqs_sym = generator(noise, training=True)
    File "/home/user/anaconda3/envs/acp-proj/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/ipykernel_384410/438853299.py", line 46, in <lambda>
        layers.Lambda(lambda x: tf.stack([x, bpf_to_blosum_layer(x)], axis=0))
    File "/tmp/ipykernel_384410/438853299.py", line 27, in bpf_to_blosum_layer
        for bpf in seqs_bpf:

    OperatorNotAllowedInGraphError: Exception encountered when calling layer 'lambda_2' (type Lambda).
    
    Iterating over a symbolic `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.
    
    Call arguments received by layer 'lambda_2' (type Lambda):
      • inputs=tf.Tensor(shape=(32, 25, 